In [ ]:
import cv2
import numpy as np

def get_output_layers(net):
    layer_names = net.getLayerNames()
    output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]
    return output_layers

def draw_prediction(img, class_id, confidence, x, y, x_plus_w, y_plus_h):
    label = str(classes[class_id])
    color = COLORS[class_id]
    cv2.rectangle(img, (x, y), (x_plus_w, y_plus_h), color, 2)
    cv2.putText(img, label, (x - 10, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

# Initialize the parameters
confThreshold = 0.5  # Confidence threshold
nmsThreshold = 0.4   # Non-maximum suppression threshold
inpWidth = 416       # Width of network's input image
inpHeight = 416      # Height of network's input image

# Load the COCO class labels our YOLO model was trained on
classesFile = "coco.names"
classes = None
with open(classesFile, 'r') as f:
    classes = [line.strip() for line in f.readlines()]

# Generate a color for each class randomly
COLORS = np.random.uniform(0, 255, size=(len(classes), 3))

# Give the configuration and weight files for the model and load the network
modelConfiguration = "yolov3.cfg"
modelWeights = "yolov3.weights"

# Load the network
net = cv2.dnn.readNetFromDarknet(modelConfiguration, modelWeights)
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)

# Tracker types
tracker_types = ['KCF', 'CSRT']
current_tracker_index = 0
current_tracker_type = tracker_types[current_tracker_index]

# Initialize the tracker
def create_tracker(tracker_type):
    if tracker_type == 'KCF':
        return cv2.TrackerKCF_create()
    elif tracker_type == 'CSRT':
        return cv2.TrackerCSRT_create()
    else:
        raise ValueError("Unsupported tracker type. Use 'KCF' or 'CSRT'.")

# Initialize the list of trackers and their class IDs
active_trackers = []
tracker_class_ids = []

# Open the video file or capture device
cap = cv2.VideoCapture("./data/cars.mp4")  # or use 0 for webcam

def perform_detection(frame):
    small_frame = cv2.resize(frame, (inpWidth, inpHeight))
    blob = cv2.dnn.blobFromImage(small_frame, 1/255, (inpWidth, inpHeight), [0, 0, 0], 1, crop=False)
    net.setInput(blob)
    outs = net.forward(get_output_layers(net))

    class_ids = []
    confidences = []
    boxes = []

    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > confThreshold:
                center_x = int(detection[0] * frame.shape[1])
                center_y = int(detection[1] * frame.shape[0])
                w = int(detection[2] * frame.shape[1])
                h = int(detection[3] * frame.shape[0])
                x = center_x - w / 2
                y = center_y - h / 2
                class_ids.append(class_id)
                confidences.append(float(confidence))
                boxes.append([x, y, w, h])

    indices = cv2.dnn.NMSBoxes(boxes, confidences, confThreshold, nmsThreshold)
    new_trackers = []
    new_tracker_class_ids = []

    if len(indices) > 0:
        for i in indices.flatten():
            box = boxes[i]
            x = box[0]
            y = box[1]
            w = box[2]
            h = box[3]
            draw_prediction(frame, class_ids[i], confidences[i], round(x), round(y), round(x + w), round(y + h))
            tracker = create_tracker(current_tracker_type)
            if tracker is not None:
                bbox = (round(x), round(y), round(w), round(h))
                tracker.init(frame, bbox)
                new_trackers.append(tracker)
                new_tracker_class_ids.append(class_ids[i])

    return new_trackers, new_tracker_class_ids

# Perform initial YOLO detection
def initial_detection():
    ret, frame = cap.read()
    if not ret:
        print("Error reading video file.")
        return [], []
    trackers, class_ids = perform_detection(frame)
    return trackers, class_ids

active_trackers, tracker_class_ids = initial_detection()

frame_counter = 0
window_name = f'Object Detection - Tracker: {current_tracker_type}'

while True:
    hasFrame, frame = cap.read()

    if not hasFrame:
        print("Done processing")
        cv2.waitKey(3000)
        break

    frame_counter += 1

    if frame_counter % 50 == 0:  # Perform detection every 50 frames
        active_trackers, tracker_class_ids = perform_detection(frame)
    else:
        updated_trackers = []
        updated_class_ids = []
        for idx, tracker in enumerate(active_trackers):
            ok, bbox = tracker.update(frame)
            if ok:
                p1 = (int(bbox[0]), int(bbox[1]))
                p2 = (int(bbox[0] + bbox[2]), int(bbox[1] + bbox[3]))
                color = COLORS[tracker_class_ids[idx] % len(COLORS)]
                cv2.rectangle(frame, p1, p2, color, 2, 1)
                label = str(classes[tracker_class_ids[idx]])
                cv2.putText(frame, label, (p1[0], p1[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
                updated_trackers.append(tracker)
                updated_class_ids.append(tracker_class_ids[idx])
        active_trackers = updated_trackers
        tracker_class_ids = updated_class_ids

    t, _ = net.getPerfProfile()
    label = tracker_types[current_tracker_index]
    cv2.putText(frame, label, (0, 15), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255))
    cv2.imshow(window_name, frame)

    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        break
    elif key == ord('t'):
        cv2.destroyWindow(window_name)
        current_tracker_index = (current_tracker_index + 1) % len(tracker_types)
        current_tracker_type = tracker_types[current_tracker_index]
        window_name = f'Object Detection - Tracker: {current_tracker_type}'
        active_trackers, tracker_class_ids = perform_detection(frame)

cap.release()
cv2.destroyAllWindows()